Demonstrations for the theory of <a class="ProveItLink" href="theory.ipynb">proveit.core_expr_types.tuples</a>
========

In [1]:
import proveit
from proveit import Lambda, ExprTuple, ExprRange, IndexedVar, safe_dummy_var, defaults
from proveit import A, a, b, f, i, j, k, l, m, n, x, y
from proveit.logic import InSet
#from proveit.logic.equality import multi_var_substitution
from proveit.numbers import Add, Mult, Neg, subtract, Natural, NaturalPos, Integer, zero, one, two
from proveit.core_expr_types import Len
from proveit.core_expr_types import (
    a_i_to_j, x_1_to_n, y_1_to_n, f_i_to_j, f_i_to_jp1)
from proveit.core_expr_types.tuples import (singular_nested_range_reduction, 
len_of_ranges_with_repeated_indices_from_1, range_from1_len, len_of_ranges_with_repeated_indices_from_1, 
len_of_ranges_with_repeated_indices, len_of_empty_range_of_ranges)
from proveit.core_expr_types.tuples import general_range_of_ranges
%begin demonstrations

In [2]:
Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, zero), one, n)).computation(assumptions=[InSet(n, NaturalPos)])

{n in NaturalPos} |- |(a_{1, 1}, ..a_{1, x}.., a_{1, 0}, ....a_{k, 1}, ..a_{k, x}.., a_{k, 0}...., a_{n, 1}, ..a_{n, x}.., a_{n, 0})| = 0

In [3]:
Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), two, n), one, zero)).computation(assumptions=[InSet(n, Natural)])

|- |(a_{1, 2}, ..a_{1, l}.., a_{1, n}, ....a_{k, 2}, ..a_{k, l}.., a_{k, n}...., a_{0, 2}, ..a_{0, l}.., a_{0, n})| = 0

In [4]:
Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, n), one, n)).computation(assumptions=[InSet(n, Natural), InSet(n, NaturalPos)])

{n in NaturalPos, n in Natural} |- |(a_{1, 1}, ..a_{1, x}.., a_{1, n}, ....a_{k, 1}, ..a_{k, x}.., a_{k, n}...., a_{n, 1}, ..a_{n, x}.., a_{n, n})| = (n * n)

In [5]:
Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, zero), two, one)).computation(assumptions=[InSet(n, Natural)])

|- |(a_{2, 1}, ..a_{2, l}.., a_{2, 0}, ....a_{k, 1}, ..a_{k, l}.., a_{k, 0}...., a_{1, 1}, ..a_{1, l}.., a_{1, 0})| = 0

In [6]:
expr_range = ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, one), one, one)

expr_range: a_{1, 1}, ..a_{1, j}.., a_{1, 1}, ....a_{i, 1}, ..a_{i, j}.., a_{i, 1}...., a_{1, 1}, ..a_{1, j}.., a_{1, 1}

In [7]:
lambda_map = Lambda((expr_range.parameter, expr_range.body.parameter), expr_range.body.body)

lambda_map: (i, j) -> a_{i, j}

In [8]:
singular_nested_range_reduction.instantiate({f:lambda_map, i:one, j:one, m:one})

|- (a_{1, 1}, ..a_{1, b}.., a_{1, 1}) = (a_{1, 1})

In [9]:
Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, one), one, one)).computation()

|- |(a_{1, 1}, ..a_{1, x}.., a_{1, 1}, ....a_{k, 1}, ..a_{k, x}.., a_{k, 1}...., a_{1, 1}, ..a_{1, x}.., a_{1, 1})| = 1

In [10]:
Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, two), one, one)).computation()

|- |(a_{1, 1}, ..a_{1, x}.., a_{1, 2}, ....a_{k, 1}, ..a_{k, x}.., a_{k, 2}...., a_{1, 1}, ..a_{1, x}.., a_{1, 2})| = 2

In [11]:
assumptions = [InSet(Add(j, Neg(i), one), Natural), 
               InSet(i, Natural), InSet(j, Natural)]

assumptions: ((j - i + 1) in Natural, i in Natural, j in Natural)

In [12]:
f_i_to_jp1

f(i), ..f(a).., f(j + 1)

In [13]:
eq = f_i_to_jp1.partition(j, assumptions)

eq: {(j - i + 1) in Natural} |- (f(i), ..f(a).., f(j + 1)) = (f(i), ..f(a).., f(j), f(j + 1))

In [14]:
eq.rhs.merger(assumptions)

{(j - i + 1) in Natural} |- (f(i), ..f(a).., f(j), f(j + 1)) = (f(i), ..f(a).., f(j + 1))

In [15]:
f_i_to_j__shift__eq = \
    f_i_to_j.shift_equivalence(new_shift=one, assumptions=[InSet(subtract(Add(j, one), i),
                                                                 Natural)])

f_i_to_j__shift__eq: {((j + 1) - i) in Natural} |- (f(i), ..f(a).., f(j)) = (f((i - 1) + 1), ..f(_a + 1).., f((j - 1) + 1))

In [16]:
f_i_to_j__shift__eq.rhs

(f((i - 1) + 1), ..f(_a + 1).., f((j - 1) + 1))

In [17]:
Add(subtract(i, one), one).simplification([InSet(i, Integer)])

{i in Integer} |- ((i - 1) + 1) = i

In [18]:
Add(Add(i, Neg(two)), two).simplification([InSet(i, Integer)])

{i in Integer} |- ((i - 2) + 2) = i

In [19]:
Add(subtract(j, one), one).simplification([InSet(j, Integer)])

{j in Integer} |- ((j - 1) + 1) = j

In [20]:
Add(subtract(j, two), two).simplification([InSet(j, Integer)])

{j in Integer} |- ((j - 2) + 2) = j

In [21]:
f_i_to_j__shift__eq.rhs.entries[0].shift_equivalence(
    old_shift=one, new_shift=two, 
    assumptions=[InSet(j, Integer), InSet(i, Integer),
                 InSet(subtract(j, subtract(i, one)), Natural)])

{(j - (i - 1)) in Natural, j in Integer, i in Integer} |- (f((i - 1) + 1), ..f(k + 1).., f((j - 1) + 1)) = (f((i + (-2)) + 2), ..f(_a + 2).., f((j + (-2)) + 2))

In [22]:
tup=ExprTuple(a, b)

tup: (a, b)

In [23]:
Len(a_i_to_j).computation([InSet(Add(j, Neg(i), one), Natural)],
                          simplify=False)

{(j - i + 1) in Natural} |- |(a_{i}, ..a_{_a}.., a_{j})| = (j - i + 1)

In [24]:
tup.prove([tup]).instantiate({a:a_i_to_j},
                            assumptions=[InSet(Add(j, Neg(i), one), Natural)])

{((a_{i}, ..a_{_a}.., a_{j}), b)} |- ((a_{i}, ..a_{_a}.., a_{j}), b)

In [25]:
eq = f_i_to_jp1.partition(k, [InSet(subtract(Add(k, one), i), Natural),
                              InSet(subtract(Add(j, one), k), Natural)])

eq: {((k + 1) - i) in Natural, ((j + 1) - k) in Natural} |- (f(i), ..f(a).., f(j + 1)) = (f(i), ..f(a).., f(k), f(k + 1), ..f(a).., f(j + 1))

In [26]:
eq.rhs.merger([InSet(subtract(Add(k, one), i), Natural),
                              InSet(subtract(Add(j, one), k), Natural)])

{((k + 1) - i) in Natural, ((j + 1) - k) in Natural} |- (f(i), ..f(a).., f(k), f(k + 1), ..f(a).., f(j + 1)) = (f(i), ..f(a).., f(j + 1))

In [27]:
eq = f_i_to_jp1.partition(i, [InSet(subtract(Add(j, one), i), Natural)])

eq: {((j + 1) - i) in Natural} |- (f(i), ..f(a).., f(j + 1)) = (f(i), f(i + 1), ..f(a).., f(j + 1))

In [28]:
eq.rhs.merger([InSet(subtract(Add(j, one), i), Natural)])

{((j + 1) - i) in Natural} |- (f(i), f(i + 1), ..f(a).., f(j + 1)) = (f(i), ..f(a).., f(j + 1))

In [29]:
eq = f_i_to_j.partition(subtract(j, one), [InSet(subtract(j, i), Natural)])

eq: {(j - i) in Natural} |- (f(i), ..f(a).., f(j)) = (f(i), ..f(a).., f(j - 1), f(j))

In [30]:
Add(subtract(j, one), one).simplification([InSet(j, Natural)])

{j in Natural} |- ((j - 1) + 1) = j

In [31]:
eq.rhs.merger([InSet(subtract(j, i), Natural), InSet(j, Natural)])

{(j - i) in Natural} |- (f(i), ..f(a).., f(j - 1), f(j)) = (f(i), ..f(a).., f(j))

In [32]:
%end demonstrations